# 数据读取

In [1]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py

import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import subgraph, add_self_loops

# === 导入模型结构 ===
from models import GraphContrastiveLearner, augment_graph, summarize_graph


# ===================== 0️⃣ 路径设置 =====================
train_graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/train_graph_with_labelmask.pt"
val_graph_path   = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/val/pyg/graph.pt"
test_graph_path  = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/test/pyg/graph.pt"

save_dir = "/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/model_save/time_morenode_high_similar_high_similar"
os.makedirs(save_dir, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ===================== 1️⃣ 加载图数据 =====================
print("\n========== 加载 train 图（带 label mask） ==========")
train_data = torch.load(train_graph_path)
summarize_graph(train_data)

print("\n========== 加载 val 图 ==========")
val_data = torch.load(val_graph_path)

print("\n========== 加载 test 图 ==========")
test_data = torch.load(test_graph_path)


# ===================== 2️⃣ 对 train_data.x 做 Z-score 标准化 =====================
x_mean = train_data.x.mean(dim=0, keepdim=True)
x_std  = train_data.x.std(dim=0, keepdim=True) + 1e-6
train_data.x = (train_data.x - x_mean) / x_std

print("🎯 已完成 train_data.x 标准化")

x_mean = test_data.x.mean(dim=0, keepdim=True)
x_std  = test_data.x.std(dim=0, keepdim=True) + 1e-6
test_data.x = (test_data.x - x_mean) / x_std

print("🎯 已完成 test_data.x 标准化")

# ===================== 3️⃣ 构造 batch 子图 =====================
batch_size = 1024
num_nodes = train_data.num_nodes

perm = torch.randperm(num_nodes)

batches = []
for i in range(0, num_nodes, batch_size):
    node_idx = perm[i:i + batch_size]

    # ---- 提取子图（必须 CPU）----
    sub_edge_index, _ = subgraph(
        node_idx,
        train_data.edge_index,
        relabel_nodes=True
    )

    sub_x = train_data.x[node_idx]

    # ---------- ⭐ 加入自环 ----------
    sub_edge_index, _ = add_self_loops(
        sub_edge_index, num_nodes=sub_x.size(0)
    )

    sub_data = Data(
        x=sub_x,
        edge_index=sub_edge_index,
    )
    batches.append(sub_data)

print(f"📌 共生成 {len(batches)} 个子图 batch")

loader = DataLoader(batches, batch_size=1, shuffle=True)


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/src

========== 加载 train 图（带 label mask） ==========


/tmp/ipykernel_118056/1724986247.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(train_graph_path)



🧩 Graph Data Summary
📊 节点数量 (num_nodes): 15000
📈 节点特征维度 (num_features): 4
🔗 边数量 (num_edges): 6000000
🔁 自环数量 (self-loops): 0
🎯 标签维度 (y_dim): torch.Size([15000])

🧾 Data对象包含字段: ['train_nolabel_mask', 'train_withlabel_mask', 'x', 'y', 'edge_index']


========== 加载 val 图 ==========


/tmp/ipykernel_118056/1724986247.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_data = torch.load(val_graph_path)



========== 加载 test 图 ==========
🎯 已完成 train_data.x 标准化
🎯 已完成 test_data.x 标准化


/tmp/ipykernel_118056/1724986247.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_data = torch.load(test_graph_path)


📌 共生成 15 个子图 batch


# STFT&CWT 特征变换

In [5]:
import torch
from torch_geometric.data import Data
import numpy as np
import os
import pandas as pd

# ====================== 0. 文件路径 ======================
train_pt_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/train_graph_with_labelmask.pt"
save_csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/train_STFT_features.csv"

# ====================== 1. 加载原始 train.pt ======================
data = torch.load(train_pt_path, map_location="cpu")

# 原始信号： (N,4)
X = data.x.cpu().numpy()      
N = len(X)

# 原始标签：假设 data.y 是 shape (N, )
labels = data.y.cpu().numpy()

print("节点数:", N)
print("原始特征维度:", X.shape)

# ====================== 2. 滑窗参数 ======================
window = 512        
step   = int(window * 0.5)    

# ====================== 3. 对每个窗口进行 STFT + 决定窗口 label ======================
stft_features = []            
window_indices = []          
window_labels  = []           # 新增：窗口最终标签

n_fft = 64
hop = 32
window_fn = torch.hann_window(n_fft)

num_windows = 0

for start in range(0, N - window + 1, step):

    # ---- 取窗口内数据 ----
    chunk = X[start:start + window]        # shape: (window,4)
    chunk_label = labels[start:start + window]  # shape: (window,)

    # ---- 统计窗口内最频繁标签 ----
    (unique, counts) = np.unique(chunk_label, return_counts=True)
    win_label = unique[np.argmax(counts)]

    window_feat = []                      

    # ---- 对每通道做 STFT ----
    for ch in range(4):
        sig = chunk[:, ch]                
        xt = torch.tensor(sig)

        stft_complex = torch.stft(
            xt,
            n_fft=n_fft,
            hop_length=hop,
            win_length=n_fft,
            window=window_fn,
            return_complex=True
        )

        stft_mag = stft_complex.abs().numpy()    
        stft_mean = stft_mag.mean(axis=1)        

        window_feat.extend(stft_mean.tolist())

    # ---- 保存结果 ----
    stft_features.append(window_feat)
    window_indices.append(start)
    window_labels.append(win_label)
    num_windows += 1


# ====================== 4. 转为 DataFrame 并保存 CSV ======================
stft_features = np.array(stft_features)    
freq_dim = stft_features.shape[1] // 4

print("窗口数量:", num_windows)
print("每窗口 STFT 特征维度:", stft_features.shape[1])

# 构建列名
columns = []
for ch in range(4):
    for f in range(freq_dim):
        columns.append(f"ch{ch+1}_freq{f}")

df = pd.DataFrame(stft_features, columns=columns)

df.insert(0, "window_start", window_indices)   # 记录窗口起点
df["label"] = window_labels                    # 新增：窗口标签

df.to_csv(save_csv_path, index=False)

print(f"✅ 已保存 STFT+标签 特征 CSV：{save_csv_path}")


/tmp/ipykernel_122805/3471273600.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(train_pt_path, map_location="cpu")


节点数: 15000
原始特征维度: (15000, 4)
窗口数量: 57
每窗口 STFT 特征维度: 132
✅ 已保存 STFT+标签 特征 CSV：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/train_STFT_features.csv


# STFT变化后的train 建图

In [8]:
def build_local_temporal_graph(
    csv_path: str,
    save_dir: str,
    num_edges: int = 10,
    label_col: int = None
):
    """
    基于时间顺序构建局部时序图。
    每一行是一个节点，上下相邻样本构成边。
    标签列可指定索引，若不指定则默认最后一列。
    🚫 自动忽略首列（常用于序号/ID），避免误入特征计算。

    参数:
        csv_path (str): 输入 CSV 文件路径。
        save_dir (str): 图结构文件的保存文件夹。
        num_edges (int): 每个节点的边数（上下平均分配）。
        label_col (int): 标签列索引（默认 None → 最后一列）。
    返回:
        (nodes_csv, edges_csv, graph_pt): 保存的文件路径元组。
    """
    os.makedirs(save_dir, exist_ok=True)

    # === 读取数据 ===
    df = pd.read_csv(csv_path)
    num_nodes = len(df)
    if num_nodes == 0:
        raise ValueError("❌ 输入 CSV 文件为空。")

    # === 标签列判断 ===
    if label_col is None:
        label_col = df.shape[1] - 1

    # === 提取标签列 ===
    y = torch.tensor(df.iloc[:, label_col].values, dtype=torch.long)

    # === 构造特征列（去掉首列 + 标签列）===
    drop_cols = [df.columns[0], df.columns[label_col]] if label_col != 0 else [df.columns[0]]
    df_features = df.drop(columns=drop_cols, errors="ignore")

    # 保留数值列
    df_features = df_features.select_dtypes(include=["float", "int"])
    if df_features.shape[1] == 0:
        raise ValueError("❌ 特征列为空，请检查输入 CSV。")

    # === 构建边 ===
    half = num_edges // 2
    edges = []
    for i in range(num_nodes):
        start_up = max(0, i - half)
        end_down = min(num_nodes, i + half + 1)
        up_neighbors = list(range(start_up, i))
        down_neighbors = list(range(i + 1, end_down))

        total_needed = num_edges
        current = len(up_neighbors) + len(down_neighbors)
        if current < total_needed:
            remaining = total_needed - current
            if i + half + 1 >= num_nodes:  # 下方不够
                extra_up = list(range(max(0, start_up - remaining), start_up))
                up_neighbors = extra_up + up_neighbors
            elif i - half < 0:  # 上方不够
                extra_down = list(range(end_down, min(num_nodes, end_down + remaining)))
                down_neighbors += extra_down

        for j in up_neighbors + down_neighbors:
            edges.append((i, j))
            edges.append((j, i))

    # === 保存节点与边 ===
    nodes_path = os.path.join(save_dir, "nodes.csv")
    edges_path = os.path.join(save_dir, "edges.csv")
    graph_path = os.path.join(save_dir, "graph.pt")

    df_features.to_csv(nodes_path, index=False)
    pd.DataFrame(edges, columns=["source", "target"]).to_csv(edges_path, index=False)

    # === 构建 PyG 图结构 ===
    edge_index = torch.tensor(edges, dtype=torch.long).T
    x = torch.tensor(df_features.values, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index, y=y)

    torch.save(data, graph_path)

    print(f"✅ 图结构构建完成，共 {num_nodes} 个节点，{len(edges)//2} 条无向边")
    print(f"📁 nodes.csv: {nodes_path}")
    print(f"📁 edges.csv: {edges_path}")
    print(f"📁 graph.pt : {graph_path}")
    print(f"🧩 特征维度: {x.shape[1]} (已自动忽略首列与标签列)")

    return nodes_path, edges_path, graph_path



# === 调用本地时序建图函数 ===
csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/train_STFT_features.csv"
pyg_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/train"
num_edges =  5
nodes_csv, edges_csv, graph_pt = build_local_temporal_graph(
    csv_path=csv_path,
    save_dir=pyg_dir,
    num_edges=num_edges
)

✅ 图结构构建完成，共 57 个节点，233 条无向边
📁 nodes.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/train/nodes.csv
📁 edges.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/train/edges.csv
📁 graph.pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/train/graph.pt
🧩 特征维度: 132 (已自动忽略首列与标签列)
